##### **목차**
<details>
<summary>CHAPTER 02 모델 평가 및 선택 29</summary>
2.1 경험 오차 및 과적합 29<br>
2.2 평가 방법 31<br>
2.3 모델 성능 측정 37<br>
2.4 비교 검증 47<br>
2.5 편향과 분산 57<br>
2.6 더 읽을거리 59<br>
연습문제 61<br>
참고문헌 62<br>
머신러닝 쉼터 64<br>
</details>

### **2.1 경험 오차 및 과적합**

##### 용어

| 용어(한글)            | 영문/표기                  | 정의                                                         | 예시                                    | 비고/주의                                    |
|----------------------|----------------------------|--------------------------------------------------------------|-----------------------------------------|----------------------------------------------|
| 오차율           | error rate                   | 전체 샘플수와 잘못 분류한 샘플 수의 비율                             | m개의 샘플중 e개의 샘플이 잘못 분류되었을 경우 -> E = a/m                      |                  |
| 정밀도           | accuracy                   | 전체 샘플수와 제대로 분류한  샘플 수의 비율                             | 1 - a/m = 1 - 오차율                      |                  |
| 오차           | error                   | 학습기의 실제 예측 값과  샘플의 실제 값 사이의 차이                             |                       |                  |
| 훈련 오차           | training error                   | 학습기가 훈련세트 상에서 만들어낸 오차                             |                       |                  |
| 일반화 오차           | generalization error                   | 학습기가 새로운 샘플 위에서 만들어낸 오차                             |                       |                  |
| 과적합           | overfitting                   | 학습기가 훈련 데이터에서 학습을 과도하게 잘하여 일반화 성능이 떨어짐                            |                       | 극복하기 어려움                 |
| 과소적합           | underfitting                   | 학습기가 훈련 데이터의 일반 성질을 제대로 배우지 못함                             |                       | 극복하기 쉬움                 |

같은 학습 알고리즘이라 하더라도, 파라미터에 따라 다른 모델로 불리기도 한다. 이때 어떤 모델을 선택해야 할까?  
일반화 오차가 가장 작은 모델을 선택하는게 이상적이지만, 우리는 일반화 오차를 직접적으로 얻을 수 없다.  
또한, 훈련 오차가 가장 작은 모델을 선택하게 된다면, 과적합의 우려가 있다.  

### **2.2 평가 방법**

우리에게 m개의 샘플을 가진 데이터 세트 D = {(x1, y1), (x2, y2), ..., (xm, ym)}가 있다고 하자.  
이때 어떻게 훈련과 평가를 수행할 수 있을까?  
일반적으로 데이터 세트 D를 적절히 분할하여 훈련 세트 S와 테스트 세트 T로 나누어 사용한다.

#### 2.2.1 홀드아웃(Hold-out)

1. 데이터 세트 D를 임의의 두 부분 집합, 즉 훈련 세트 S와 테스트 세트 T로 나눈다.  
   (가능하면 데이터 분포를 유지하도록 나누는 것이 바람직하다.)
2. 훈련 세트 S로 학습한 모델의 성능을 테스트 세트 T로 측정하여 일반화 오차에 대한 추정치를 얻는다.

##### 실습

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from collections import Counter

data = load_iris()
x = data.data
y = data.target
np.random.seed(42)
indices = np.arange(len(x))
np.random.shuffle(indices)
x, y = x[indices],  y[indices]

In [4]:
def hold_out(split_ratio = 0.7):
    split_idx = int(len(x) * split_ratio)
    x_train, x_test = x[:split_idx], x[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return x_train, x_test, y_train, y_test

In [5]:
# 간단한 검증
x_train, x_test, y_train, y_test = hold_out(split_ratio=0.7)
majority_class = Counter(y_train).most_common(1)[0][0] # 최빈 클래스 추출
y_pred = np.full(len(y_test), majority_class) # 모든 테스트 샘플을 최빈 클래스로 예측
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"[Hold-out] Accuracy: {accuracy:.4f}")

[Hold-out] Accuracy: 0.2222


#### 2.2.2 교차 검증(Cross-validation)

1. 데이터 세트 D를 k개의 서로소 부분 집합으로 나눈다.  
   (역시 데이터 분포를 유지하도록 나누는 것이 바람직하다.)
2. 각 반복에서 k-1개의 부분집합을 훈련 세트로 사용하고, 나머지 1개의 부분집합을 테스트 세트로 사용한다.
3. 이렇게 k개의 훈련/테스트 세트를 만들어 k번 훈련과 테스트를 수행한 뒤, k개의 테스트 결과의 평균을 계산해 모델 성능을 평가한다.

교차 검증 결과의 안정성과 정확도는 k의 값에 따라 달라진다.  
이를 k겹 교차 검증(k-fold cross-validation)이라고 부른다.

##### 실습

In [6]:
def cross_validation(k = 5):
    fold_size = len(x) // k
    accuracies = [] 
    for i in range(k):
        start = i * fold_size
        end = (i + 1) * fold_size if i < k else len(x)

        x_test = x[start:end]
        y_test = y[start:end]

        x_train = np.concatenate([x[:start], x[end:]], axis = 0)
        y_train = np.concatenate([y[:start], y[end:]], axis = 0)

        majority_class = Counter(y_train).most_common(1)[0][0] # 최빈 클래스 추출
        y_pred = np.full(len(y_test), majority_class) # 모든 테스트 샘플을 최빈 클래스로 예측
        accuracy = np.sum(y_pred == y_test) / len(y_test)

        accuracies.append(accuracy)
    print(f"[Cross-validation] Accuracies: {accuracies}")
    print(f"[Cross-validation] Mean accuracy: {np.mean(accuracies):.4f}")

In [12]:
cross_validation(k = 5)

[Cross-validation] Accuracies: [np.float64(0.3), np.float64(0.23333333333333334), np.float64(0.26666666666666666), np.float64(0.26666666666666666), np.float64(0.23333333333333334)]
[Cross-validation] Mean accuracy: 0.2600


#### 2.2.3 부트스트래핑(Bootstrapping)

홀드아웃이나 교차 검증 방식은 일부 샘플을 테스트용으로 제외하므로 훈련에 모든 샘플을 활용할 수 없다는 단점이 있다.  
만약 데이터 세트 D의 모든 샘플을 훈련에 사용하고 싶다면, 부트스트래핑 기법이 유용하다.

1. 데이터 세트 D에는 m개의 샘플이 있다. 새로운 데이터 세트 D'를 비워둔 상태로 시작한다.
2. D에서 무작위로 샘플 하나를 선택하여 D'에 복사한다. 이때 같은 샘플이 중복해서 선택될 수 있다.
3. 이 과정을 m번 반복하면, D'에는 m개의 샘플이 들어가게 된다.
4. 수학적으로, m번 복원추출을 할 때 샘플이 한 번도 뽑히지 않을 확률은  
   (1 - 1/m)^m ≈ e^-1 ≈ 36.8%이다.  
   즉, 데이터 세트 D의 약 36.8%의 샘플은 D'에 포함되지 않는다.
5. 이렇게 만들어진 D'를 훈련 세트로 사용하고, D에서 선택되지 않은 샘플들을 테스트 세트로 사용한다.

부트스트래핑의 특징:
- 데이터 양이 적거나, 훈련/테스트 분할이 어려울 때 유용하다.
- 앙상블 학습 기법(예: 배깅)과 잘 어울린다.
- 단점으로, D'의 데이터 분포가 원본 D와 다를 수 있어 편향이 발생할 수 있다.
- 따라서 데이터가 풍부하다면 홀드아웃이나 교차 검증을 더 자주 사용한다.

##### 실습

In [13]:
def bootstrapping():
    m = len(x)
    bootstrap_indices = np.random.choice(np.arange(m), size = m, replace=True)
    oob_indices = np.setdiff1d(np.arange(m), np.unique(bootstrap_indices))

    x_train = x[bootstrap_indices]
    x_test = x[oob_indices]
    y_train = y[bootstrap_indices]
    y_test = y[oob_indices]
    
    majority_class = Counter(y_train).most_common(1)[0][0] # 최빈 클래스 추출
    y_pred = np.full(len(y_test), majority_class) # 모든 테스트 샘플을 최빈 클래스로 예측
    accuracy = np.sum(y_pred == y_test) / len(y_test)

    print(f"[Bootstrap] 선택되지 않은 샘플 비율 : {(len(oob_indices) / m):.4f}")
    # 정확도
    if len(y_test) > 0:
        accuracy = np.sum(y_pred == y_test) / len(y_test)
        print(f"[Bootstrap] Accuracy: {accuracy:.4f}")
    else:
        print("[Bootstrap] OOB 샘플이 없어 정확도를 계산할 수 없습니다.")

In [14]:
bootstrapping()

[Bootstrap] 선택되지 않은 샘플 비율 : 0.4000
[Bootstrap] Accuracy: 0.3333


#### **2.3 모델 성능 측정**

#### **2.4 비교 검증**

#### **2.5 편향과 분산**